In [1]:
import numpy as np

In [159]:
class blackjack_game:
    def __init__(self, dealer_decision_point):
        self.dealer_decision_point = dealer_decision_point
        self.cards = ['Ace', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'face']
        
    def drawtwocards(self):
        rand_cards = np.random.randint(0, len(self.cards), size=2)
        return rand_cards
    
    def drawonecard(self):
        rand_card = np.random.randint(0, len(self.cards))
        return rand_card
    
    def computepoints(self, rand_cards):
        points = 0
        for i in rand_cards:
            if type(self.cards[i]) is int:
                points += self.cards[i]
            elif self.cards[i] is 'face':
                points += 10
            elif self.cards[i] is 'Ace':
                if points <= 10:
                    points += 10
                elif points > 10:
                    points += 1
        return points
    
    def dealerplay(self, point_total):
        if point_total < self.dealer_decision_point:
            return 'hit'
        elif point_total >= self.dealer_decision_point:
            return 'stick'
        
    def agent_obs(self, agent_cards, dealer_cards):
        a_cards = [self.cards[i] for i in agent_cards]
        d_card = self.cards[dealer_cards[0]]
        a_points = self.computepoints(a_cards)
        d_points = self.computepoints([d_card])
        agent_ace_usable = int(a_points <= 10)
        return a_points, d_points, agent_ace_usable
    
    def check_win(self, agent_cards, dealer_cards, both_done_playing):
        agent_points = self.computepoints(agent_cards)
        dealer_points = self.computepoints(dealer_cards)
        agent_win, dealer_win = 0, 0
        if agent_points == 21: agent_win += 1
        if dealer_points == 21: dealer_win += 1
        
        if both_done_playing and agent_points > dealer_points:
            agent_win += 1
        if both_done_playing and dealer_points > agent_points:
            dealer_win += 1
            
        if agent_win == dealer_win: return 0
        if agent_win > dealer_win: return 1
        if agent_win < dealer_win: return -1
        
        else:
            return None
        

In [161]:
game = blackjack_game(17)
a_cards, d_cards = game.drawtwocards(), game.drawtwocards()
game.check_win(a_cards, d_cards, True)

-1

In [162]:
print(a_cards, d_cards)

[1 7] [3 6]
